# Create Teaching labels

In [ ]:
import models.model_student as student

In [ ]:
import tensorflow as tf
all_files = tf.data.Dataset.list_files("runs2\\*\\.fullpic\\*.png")
files = list(all_files.take(1203).as_numpy_iterator())

In [ ]:
files[:5]

In [588]:
from PIL import Image
import models.model_experiemnt_lab as teacher
import numpy as np
def file_to_pred(f, T=1):
    im = Image.open(f)
    im = im.resize((320,200))
    preds = teacher.make_preds(im)
    # preds = np.exp(preds/T)/sum(np.exp(preds/T))
    return preds

In [589]:
from tqdm.notebook import tqdm
import threading
T=1
def get_raw_preds():
    global raw_preds
    raw_preds = [file_to_pred(i,T) for i in tqdm(files)]
thread = threading.Thread(target=get_raw_preds)
thread.start()

  0%|          | 0/1203 [00:00<?, ?it/s]

In [590]:
len(raw_preds)

1203

In [592]:
soft_preds = {i:[np.exp(p[:,0]/i)/sum(np.exp(p[:,0]/i)) for p in raw_preds] for i in [0.05, 0.1, 0.5, 1, 1.5]}
sum_preds = [p[:,0]/sum(p[:,0]) for p in raw_preds]

In [597]:
import pandas as pd
dfs = {i:pd.DataFrame({f: p for f,p in zip(files, soft_preds[i])}) for i in soft_preds.keys()}
dfs['sum_normal'] = pd.DataFrame({f: p for f,p in zip(files, sum_preds)})
dfs['raw'] = pd.DataFrame({f: p[:,0] for f,p in zip(files, raw_preds)})

In [598]:
dfs['raw']

,runs2\\00102\\.fullpic\\0000000004+system.png,runs2\\00317\\.fullpic\\0000000002+system.png,runs2\\00017\\.fullpic\\0000000032+system.png,runs2\\00163\\.fullpic\\0000000078+system.png,runs2\\00235\\.fullpic\\0000000054+system.png,runs2\\00474\\.fullpic\\0000000038+system.png,runs2\\00115\\.fullpic\\0000000063+system.png,runs2\\00341\\.fullpic\\0000000010+system.png,runs2\\00145\\.fullpic\\0000000025+system.png,runs2\\00054\\.fullpic\\0000000013+system.png,...,runs2\\00369\\.fullpic\\0000000011+system.png,runs2\\00396\\.fullpic\\0000000001+system.png,runs2\\00147\\.fullpic\\0000000062+system.png,runs2\\00412\\.fullpic\\0000000008+system.png,runs2\\00536\\.fullpic\\0000000043+system.png,runs2\\00001\\.fullpic\\0000000011+system.png,runs2\\00163\\.fullpic\\0000000074+system.png,runs2\\00010\\.fullpic\\0000000016+system.png,runs2\\00525\\.fullpic\\0000000046+system.png,runs2\\00394\\.fullpic\\0000000008+system.png
0,0.087301,0.222189,0.599788,0.985794,0.999261,0.906918,0.980780,0.283804,0.822787,0.999997,...,0.911989,0.634173,0.905701,0.660436,0.662994,0.713454,0.892405,0.999891,0.789046,0.975602
1,0.124052,0.042333,0.553326,0.996808,0.764664,0.970123,0.870806,0.935816,0.957210,0.999932,...,0.904266,0.992675,0.993889,0.470999,0.728106,0.472588,0.960322,0.043001,0.745661,0.998113
2,0.066871,0.028930,0.870369,0.949255,0.752066,0.972466,0.856692,0.982205,0.974445,0.767058,...,0.246189,0.999999,0.999898,0.144189,0.834711,0.165992,0.919098,0.775293,0.903880,0.999974
3,0.073110,0.081679,0.461081,0.594493,0.688664,0.922850,0.775210,0.922588,0.890517,0.639912,...,0.747838,0.989484,0.472735,0.042126,0.305730,0.344170,0.825091,0.982653,0.986104,0.992600
4,0.069681,0.060287,0.540846,0.834478,0.617845,0.923751,0.470351,0.976705,0.908055,0.302170,...,0.673010,0.972093,0.979582,0.316141,0.597016,0.448714,0.943219,0.911375,0.971588,0.990894
5,0.149883,0.060169,0.783717,0.597894,0.974850,0.936873,0.967320,0.973124,0.993774,0.988679,...,0.604072,0.192377,0.998003,0.139393,0.875005,0.403869,0.869902,0.961350,0.902294,0.962599


In [ ]:
dfs[0.05][files[0]], dfs['sum_normal'][files[0]]

In [599]:
import os
par_dir = 'teacher_preds'
os.makedirs(par_dir, exist_ok=True)
for i in dfs.keys():
    dfs[i].to_csv(os.path.join(par_dir, f"{i}.csv"), index=False)

# Read Teaching Labels

In [586]:
import pandas as pd
import os
teach_par = 'teacher_preds'
csvs = [os.path.join(teach_par, i) for i in os.listdir(teach_par)]
dfs = {csv.split(os.sep)[-1][:-4]: pd.read_csv(csv) for csv in csvs}

In [587]:
from sklearn.model_selection import train_test_split
files = dfs["1"].columns
train_files, val_files = train_test_split(files, test_size=0.2)
len(files), len(train_files), len(val_files)

(1203, 962, 241)

In [3]:
import tensorflow as tf
from math import ceil
# def batched(gen, batch_size):
#     gen = iter(gen)
#     def g():
#         nxt = []
#         try:
#             for _ in range(batch_size):
#                 nxt.append(next(gen))
#         except StopIteration as e:
#             if len(nxt)==0:
#                 raise e
#         nxt = list(zip(*nxt))
#         nxt = [np.array(i) for i in nxt]
#         yield nxt
#     return g
class StudentTrainSequence(tf.keras.utils.Sequence):
    def __init__(self, frame, batch_size) -> None:
        self.frame = frame
        self.batch_size = batch_size
    def __len__(self):
        return ceil(self.frame.shape[1]/self.batch_size)
    def __getitem__(self, idx):
        low = idx*self.batch_size
        high = min(low + self.batch_size, self.frame.shape[1])
        batch_files = self.frame.columns[low:high]
        batch_y = self.frame.loc[:, batch_files].to_numpy().T
        batch_x = batch_files
        batch_x = [tf.io.read_file(f) for f in batch_x]
        batch_x = [tf.image.resize(tf.io.decode_png(f), (200,320)) for f in batch_x]
        batch_x = [tf.image.convert_image_dtype(img, tf.float32) for img in batch_x]
        batch_x = tf.stack(batch_x)
        return batch_x, batch_y

In [600]:
seq = StudentTrainSequence(dfs["1"], 3)
i,j = seq[0]
i.shape, j.shape

KeyError: '1'

In [4]:
train_dfs = {i: dfs[i].loc[:, train_files] for i in dfs.keys()}
val_dfs = {i: dfs[i].loc[:, val_files] for i in dfs.keys()}
train_seq = {i: StudentTrainSequence(train_dfs[i], 100) for i in train_dfs.keys()}
val_seq = {i: StudentTrainSequence(val_dfs[i], 100) for i in val_dfs.keys()}

In [5]:
import models.model_student as student
student.model.summary()

Loading Tensorflow: Done    
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 196, 316, 32)      2432      
                                                                 
 max_pooling2d (MaxPooling2  (None, 39, 63, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 35, 59, 32)        25632     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 7, 11, 32)         0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 2464)              0         
                                                                 
 dense (Dense)             

In [6]:
student_par = "student_models"

In [7]:
os.makedirs(student_par, exist_ok=True)
student.model.save_weights(os.path.join(student_par, 'blank.keras'))

In [ ]:
student.model.load_weights(os.path.join(student_par, 'blank.keras'))
student.model.compile("adam",
              tf.keras.losses.CategoricalCrossentropy(),
              jit_compile=True)

In [13]:
all_history = None
for epoch in range(10):
    for k in train_seq.keys():
        name = f'Temp_{k}'
        print(f'Model: {name}')
        load_file = 'blank.keras' if epoch==0 else f'{k}_e{epoch-1:0=2}.keras'
        student.model.load_weights(os.path.join(student_par, load_file))
        student.model.compile("adam",
                    tf.keras.losses.CategoricalCrossentropy(),
                    jit_compile=True)
        hist = student.model.fit(
            train_seq[k],
            validation_data=val_seq[k],
            epochs=epoch+1,
            initial_epoch=epoch
        )
        student.model.save_weights(os.path.join(student_par, f'{k}_e{epoch:0=2}.keras'))
        hist.history['model'] = [name]
        hist.history['epoch'] = [epoch+1]
        history = pd.DataFrame(hist.history)
        if all_history is None:
            all_history = history
        else:
            all_history = pd.concat([all_history, history])
        all_history.to_csv('student_history.csv',index=False)


Model: Temp_0.05
10/10 [==============================] - 65s 6s/step - loss: 69.2934 - val_loss: 1.8037
Model: Temp_0.1
10/10 [==============================] - 61s 6s/step - loss: 65.4549 - val_loss: 1.7936
Model: Temp_0.5
10/10 [==============================] - 61s 6s/step - loss: 72.3306 - val_loss: 1.7949
Model: Temp_1.5
10/10 [==============================] - 59s 6s/step - loss: 66.7552 - val_loss: 1.8036
Model: Temp_1
10/10 [==============================] - 60s 6s/step - loss: 57.3983 - val_loss: 1.8002
Model: Temp_sum_normal
10/10 [==============================] - 56s 5s/step - loss: 65.9213 - val_loss: 1.8020
Model: Temp_0.05
Epoch 2/2
10/10 [==============================] - 57s 5s/step - loss: 1.7999 - val_loss: 1.7906
Model: Temp_0.1
Epoch 2/2
10/10 [==============================] - 56s 5s/step - loss: 1.7955 - val_loss: 1.7910
Model: Temp_0.5
Epoch 2/2
10/10 [==============================] - 58s 5s/step - loss: 1.7949 - val_loss: 1.7919
Model: Temp_1.5
Epoch 2/2
10/1

In [15]:
list(tf.data.Dataset.list_files("student_models\\*_e09.keras").as_numpy_iterator())

[b'student_models\\1_e09.keras',
 b'student_models\\sum_normal_e09.keras',
 b'student_models\\0.05_e09.keras',
 b'student_models\\0.1_e09.keras',
 b'student_models\\1.5_e09.keras',
 b'student_models\\0.5_e09.keras']

In [38]:
import numpy as np
res = student.model(np.random.random((1,200,320,3)))
tf.argmax(res[0]).numpy()

5

In [520]:
student.model.load_weights("student_models\\blank.keras")

In [525]:
student.model(np.random.random((1,200,320,3))*10000000)

<tf.Tensor: shape=(1, 6), dtype=float32, numpy=array([[0., 1., 0., 0., 0., 0.]], dtype=float32)>

In [278]:
model = tf.keras.Sequential(layers=[
    tf.keras.layers.Conv2D(32, (5,5), input_shape=(200,320,3), activation='relu'),
    tf.keras.layers.MaxPool2D((5,5)),
    tf.keras.layers.Conv2D(32, (5,5), activation='relu'),
    tf.keras.layers.MaxPool2D((5,5)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1000, 'relu'),
    tf.keras.layers.Dense(1000, 'relu'),
    tf.keras.layers.Dense(100, 'relu'),
    tf.keras.layers.Dense(6, 'softmax'),
])

In [526]:
model.load_weights("student_models\\blank.keras")

In [584]:
model(np.random.random((1,200,320,3)))

<tf.Tensor: shape=(1, 6), dtype=float32, numpy=
array([[0.15038343, 0.20452508, 0.15909198, 0.14421031, 0.17543429,
        0.16635491]], dtype=float32)>